In [1]:
import shap, keras
import numpy as np
import tensorflow as tf
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=1000)
X_train = sequence.pad_sequences(X_train, maxlen=100).astype("float64")
X_test = sequence.pad_sequences(X_test, maxlen=100).astype("float64")
# create the model
embedding_vector_length = 32
inp = keras.layers.Input(shape=(100,))
mult = keras.layers.Multiply()([inp, inp])
out = keras.layers.Dense(1)(mult)
model = keras.models.Model(inputs=[inp], outputs=out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
multiply_1 (Multiply)           (None, 100)          0           input_1[0][0]                    
                                                                 input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1)            101         multiply_1[0][0]                 
Total params: 101
Trainable params: 101
Non-trainable params: 0
__________________________________________________________________________________________________
None


In [2]:
inds = np.random.choice(X_train.shape[0], 3, replace=False)
data = X_train[inds,:]
test_in = X_test[10:11,:]
e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].input), data)
shap_values = e.shap_values(test_in)
sums = np.array([shap_values[i].sum() for i in range(len(shap_values))])
sess = tf.keras.backend.get_session()
diff = sess.run(model.layers[-1].input, feed_dict={model.layers[0].input: test_in})[0,:] - \
    sess.run(model.layers[-1].input, feed_dict={model.layers[0].input: data}).mean(0)
assert np.allclose(sums, diff, atol=1e-06), "Sum of SHAP values does not match difference!"